In [1]:
''' Use a pretrained neural network (VGG16) to extract features of picture, and use clustering algorithm (KMeans) to
group parts of the picture with similar features '''

import cv2
import numpy as np


def get_training_data(folder_name, s_pic_diameter, num, img, greyscale):
    ''' Creates a folder with small pictures for training, and returns its contents
    
    Parameters
    ----------
    folder_name: str
        path to folder where pictures will go
    
    s_pic_diameter: int
        diameter of small pictures (square)
    
    num: int
        number of pictures to create, before some are weeded out for being outside of picture range
    
    img: np.ndarray
        the large image over which training and testing occurs
    '''
    
    dim = 3
    
    import random
    import ImageSampler
    import glob

    print('Creating training data...')
    
    #put random sections of overall picture into folder
    for i in range(0,num):
        x=random.randint(0,np.size(img,1)-s_pic_diameter)
        y=random.randint(0,np.size(img,0)-s_pic_diameter)
        temp_pic = img[y:(y+s_pic_diameter), x:(x+s_pic_diameter)]
        cv2.imwrite(folder_name+str(i)+'.png', temp_pic)
    
    #remove images that are partly black, in case are outside of the picture
    ImageSampler.remove_black(folder_name, s_pic_diameter, s_pic_diameter, flag=1)

    #load these pictures into variable train_images
    files = glob.glob(folder_name+'*png')
    train_images = np.zeros((len(files), s_pic_diameter, s_pic_diameter, dim))
    counter=0
    for file in files:
        temp_img = cv2.imread(file, greyscale)
        train_images[counter] = temp_img
        counter += 1
    
    #return variable of pictures
    return train_images


def get_testing_data(l_pic_diameter, s_pic_diameter, img, x, y, final_folder):
    ''' Returns testing data in the form of a collection of small pictures surrounding each pixel in a large picture
    and also puts original image into folder into which KMeans visualization will ultimately go
    
    Parameters
    ----------
    l_pic_diameter: int
        diameter of large picture for testing
    
    s_pic_diameter: int
        diameter of small picture that will surround every pixel to determine what group it goes in to
    
    img: np.ndarray
        the large image over which training and testing occurs
        
    x: int
        pixel of starting x value for the picture subset
    
    y: int
        pixel of starting y value for the picture subset
    
    final_folder: str
        folder to put this image into
    '''
    
    print('Creating testing data...')
    dim = 3
    
    #get image subset
    img_subset = img[y:y + l_pic_diameter, x:x + l_pic_diameter]
    test_img = np.zeros((l_pic_diameter*l_pic_diameter, s_pic_diameter, s_pic_diameter, dim))
    
    #put image into folder
    cv2.imwrite(final_folder+'original.png', img_subset)
    
    #loop through pixels and extract the surrounding image for each pixel
    counter=0
    for row in range(0,l_pic_diameter):
        for col in range(0,l_pic_diameter):
            temp_img = img[y+row-int(s_pic_diameter/2+0.5):y+row+int(s_pic_diameter/2),
                           x+col-int(s_pic_diameter/2+0.5):x+col+int(s_pic_diameter/2)]
            test_img[counter] = temp_img
            counter += 1
    
    #return collection of small pictures
    return test_img

def network_output(model, output_layer, images):
    ''' Preprocesses and trains network on training data
    
    Parameters
    ----------
    model: model (?)
        the pretrained network you are using
    
    output_layer: int
        the intermediate layer where you want output
    
    images: np.array (?)
        the list of images being used for training or testing
    '''
    
    from keras.applications.vgg16 import preprocess_input
    from keras import backend as K
    
    print('Getting network output...')
    
    #preprocess data
    images = np.array(images)
    images = preprocess_input(images)
    
    #get output
    get_layer_output = K.function([model.layers[0].input], [model.layers[output_layer].output])
    output = get_layer_output([images])[0]
    
    #return output in an appropriate shape for clustering algorithms
    return output.reshape(output.shape[0], np.prod(output.shape[1:]))

def run_KMeans(train_output, test_output, number_clusters):
    ''' Returns the fit of the KMeans algorithm of the testing data after training on the training data
    
    Parameters
    ----------
    train_output: np.array
        array of training images
    
    test_output: np.array
        array of testing images
    
    number_clusters: int
        number of clusters to get out of KMeans algorithm
    '''
    
    from sklearn.cluster import KMeans
    
    print('Running KMeans...')
    
    #train KMeans
    kmeans = KMeans(n_clusters = number_clusters, random_state=0)
    kmeans.fit(train_output)
    
    #return prediction from KMeans
    return kmeans.predict(test_output)

def visualize_KMeans(fit, l_pic_diameter, folder_name):
    ''' Writes image visualizing KMeans output into the specified folder
    
    Parameters
    ----------
    fit: np.array (?)
        the output of KMeans prediction from testing data
    
    l_pic_diameter: int
        diameter of square picture over which testing occurred
    
    folder_name: str
        name of folder into which KMeans visualization should be written
    '''
    
    print('Drawing output...')
    dim = 3
    
    #create picture
    newpic = np.zeros((l_pic_diameter,l_pic_diameter,dim), dtype=np.uint8)
    
    #draw image onto picture, with different colors representing different KMeans groups
    n = 0
    for row in range(0,l_pic_diameter):
        for col in range(0, l_pic_diameter):
            if fit[n] < 2:
                newpic[row][col] = [255 - 130*(1-fit[n]),0,0]
            elif fit[n] < 4:
                newpic[row][col] = [0,0,255 - 130*(3-fit[n])]
            else:
                newpic[row][col] = [0,255 - 130*(5-fit[n]),0]
            n += 1

    cv2.imwrite(folder_name+'test.png', newpic)
    print('Done.')

In [ ]:
#run program to separate features

from keras.applications.vgg16 import VGG16

#fill in variables
train_path = '/Users/dbasili/drone_maps/riverside012_transparent_mosaic_group1.tif'
folder_name = '/Users/dbasili/train_images/'
final_folder_name = '/Users/dbasili/drone_maps/'
l_pic_diameter = 450            #diameter of the large picture for testing
s_pic_diameter = 9              #diameter of each smaller picture, preferrably odd
num = 10000                     #number of training pictures (before some are weeded out for being outside picture)
starting_x = 13300              #starting coordinates of testing picture
starting_y = 1900
output_layer = 10               #layer of output from pretrained neural network
number_clusters = 3             #number of clusters from KMeans
greyscale = 1                   #read image in color or greyscale; 0 = greyscale, 1 = color

#get image
img = cv2.imread(train_path, greyscale)

# get pictures into variable called files
train_images = get_training_data(folder_name, s_pic_diameter, num, img, greyscale)
test_images = get_testing_data(l_pic_diameter, s_pic_diameter, img, starting_x, starting_y, final_folder_name)

#create model for training data
model = VGG16(weights='imagenet', include_top=False)

#get output from training and testing data
training_output = network_output(model, output_layer, train_images)
testing_output = network_output(model, output_layer, test_images)

#get and visualize KMeans prediction
fit = run_KMeans(training_output, testing_output, number_clusters)
visualize_KMeans(fit, l_pic_diameter, final_folder_name)